# DynamoDB Hands-On Lab

Create a table, store documents with different attributes, query with both the API and PartiQL, load into DataFrames, and clean up.

## Setup — Get Your AWS Credentials

You need two values from your local machine to paste into **Colab Secrets**.

### Find your credentials
Open a terminal and run:
```
cat ~/.aws/credentials
```
You should see:
```
[default]
aws_access_key_id = AKIA...
aws_secret_access_key = Y+Co...
```

### Add secrets to Colab
1. Click the **🔑 key icon** in the left sidebar
2. Click **+ Add new secret** and create these two:
   - `AWS_ACCESS_KEY_ID` — starts with `AKIA...` (**required**)
   - `AWS_SECRET_ACCESS_KEY` — long string of letters/numbers (**required**)
3. Toggle **Notebook access** ON for each secret
4. Run the cell below — it should print `Connected as: arn:aws:sts::...`

In [ ]:
!pip install -q boto3

import boto3, os
from decimal import Decimal
from botocore.exceptions import ClientError

try:
    from google.colab import userdata
    os.environ['AWS_ACCESS_KEY_ID'] = userdata.get('aws_access_key_id')
    os.environ['AWS_SECRET_ACCESS_KEY'] = userdata.get('aws_secret_access_key')
    print('Loaded credentials from Colab Secrets')
except ImportError:
    print('Not in Colab - using default AWS credential chain')

sts = boto3.client('sts', region_name='us-east-1')
identity = sts.get_caller_identity()
print(f"Connected as: {identity['Arn']}")
print(f"Account: {identity['Account']}")

REGION = 'us-east-1'
TABLE_NAME = 'student-directory-lab'
dynamodb = boto3.resource('dynamodb', region_name=REGION)
print(f'DynamoDB ready (region: {REGION})')

## Step 1: Create a Table
We define only the partition key (`student_id`). No columns for name, gpa, etc.
Different items can have completely different attributes - that is the document model.

In [ ]:
try:
    table = dynamodb.create_table(
        TableName=TABLE_NAME,
        KeySchema=[{'AttributeName': 'student_id', 'KeyType': 'HASH'}],
        AttributeDefinitions=[{'AttributeName': 'student_id', 'AttributeType': 'S'}],
        BillingMode='PAY_PER_REQUEST'
    )
    print(f'Creating table "{TABLE_NAME}"...')
    table.wait_until_exists()
    print(f'Table created! Status: {table.table_status}')
except ClientError as e:
    if e.response['Error']['Code'] == 'ResourceInUseException':
        print(f'Table "{TABLE_NAME}" already exists.')
        table = dynamodb.Table(TABLE_NAME)
    else: raise
print(f'ARN: {table.table_arn}')

## Step 2: Put Documents
Each item is a JSON document. Notice each student has **different attributes**:
- Alice has `contact` (nested map)
- Bob has `thesis_title`
- Carol has `scholarship`
- David has `minor`

No ALTER TABLE needed. Just include whatever attributes are relevant.

In [ ]:
students = [
    {'student_id': 'stu-001', 'name': 'Alice Johnson', 'major': 'Data Science',
     'year': 3, 'gpa': Decimal('3.85'), 'skills': ['python', 'sql', 'statistics'],
     'contact': {'email': 'alice@example.edu', 'campus': 'main'}},
    {'student_id': 'stu-002', 'name': 'Bob Martinez', 'major': 'Computer Science',
     'year': 4, 'gpa': Decimal('3.72'), 'skills': ['java', 'python', 'aws'],
     'thesis_title': 'Distributed Cache Invalidation'},
    {'student_id': 'stu-003', 'name': 'Carol Chen', 'major': 'Data Science',
     'year': 2, 'gpa': Decimal('3.95'), 'skills': ['r', 'python', 'tableau'],
     'scholarship': True},
    {'student_id': 'stu-004', 'name': 'David Kim', 'major': 'Applied Mathematics',
     'year': 3, 'gpa': Decimal('3.60'), 'skills': ['python', 'matlab'],
     'minor': 'Data Science'}
]

for s in students:
    table.put_item(Item=s)
    common = {'student_id', 'name', 'major', 'year', 'gpa', 'skills'}
    unique = set(s.keys()) - common
    extra = f'  (unique: {unique})' if unique else ''
    print(f'Created: {s["name"]}{extra}')

print(f'\nStored {len(students)} documents with different attribute shapes')

## Step 3: Get an Item
Fast lookup by partition key. SQL equivalent: `SELECT * FROM students WHERE student_id = 'stu-001'`

In [ ]:
response = table.get_item(Key={'student_id': 'stu-001'})
if 'Item' in response:
    item = response['Item']
    print(f'Name:    {item["name"]}')
    print(f'Major:   {item["major"]}')
    print(f'GPA:     {float(item["gpa"])}')
    print(f'Skills:  {item["skills"]}')
    print(f'Contact: {item.get("contact", "N/A")}')

# Non-existent item returns empty, not an error
resp2 = table.get_item(Key={'student_id': 'stu-999'})
print(f'\nstu-999 found: {"Item" in resp2}')

## Step 4: Update Attributes
Unlike `put_item` (replaces entire item), `update_item` changes specific attributes.
The expression syntax is verbose but prevents injection and handles reserved words.

In [ ]:
# Update year and GPA
print('Updating Alice: year 3->4, GPA 3.85->3.90')
table.update_item(
    Key={'student_id': 'stu-001'},
    UpdateExpression='SET #yr = :y, gpa = :g',
    ExpressionAttributeNames={'#yr': 'year'},
    ExpressionAttributeValues={':y': 4, ':g': Decimal('3.90')}
)
item = table.get_item(Key={'student_id': 'stu-001'})['Item']
print(f'After: year={item["year"]}, gpa={float(item["gpa"])}')

# Add a NEW attribute (no ALTER TABLE needed!)
print('\nAdding honors attribute...')
table.update_item(
    Key={'student_id': 'stu-001'},
    UpdateExpression='SET honors = :h',
    ExpressionAttributeValues={':h': True}
)

# Append to a list
print('Appending "aws" to skills...')
table.update_item(
    Key={'student_id': 'stu-001'},
    UpdateExpression='SET skills = list_append(skills, :new)',
    ExpressionAttributeValues={':new': ['aws']}
)
item = table.get_item(Key={'student_id': 'stu-001'})['Item']
print(f'Skills: {item["skills"]}')

## Step 5: Scan
Scan reads **every** item, then filters. Fine for small tables, expensive for millions of rows.

In [ ]:
from boto3.dynamodb.conditions import Attr

print('All items:')
for item in table.scan()['Items']:
    print(f'  {item["student_id"]}: {item["name"]}  attrs={list(item.keys())}')

print('\nData Science students:')
resp = table.scan(FilterExpression=Attr('major').eq('Data Science'))
for item in resp['Items']:
    print(f'  {item["name"]} (GPA: {float(item["gpa"])})')

print('\nGPA > 3.80:')
resp = table.scan(FilterExpression=Attr('gpa').gt(Decimal('3.80')))
for item in resp['Items']:
    print(f'  {item["name"]} (GPA: {float(item["gpa"])})')

## Step 6: PartiQL — SQL Syntax for DynamoDB

DynamoDB also supports **PartiQL**, a SQL-compatible query language. This lets you write familiar `SELECT`, `INSERT`, and `UPDATE` statements instead of using the API.

**Key difference:** PartiQL uses the **low-level client** (`boto3.client`), not the resource (`boto3.resource`). The responses use DynamoDB's type descriptors (`{'S': 'value'}` instead of just `'value'`).

In [ ]:
# PartiQL uses the low-level CLIENT, not the resource
client = boto3.client('dynamodb', region_name=REGION)

# SELECT one item by partition key
print('=== PartiQL: Get one student ===')
response = client.execute_statement(
    Statement=f'SELECT * FROM "{TABLE_NAME}" WHERE student_id=?',
    Parameters=[{'S': 'stu-001'}]
)
for item in response['Items']:
    print(f"  {item['name']['S']} — {item['major']['S']} — GPA: {item['gpa']['N']}")

# SELECT with a filter (works like a WHERE clause)
print('\n=== PartiQL: Data Science students ===')
response = client.execute_statement(
    Statement=f'SELECT name, gpa FROM "{TABLE_NAME}" WHERE major=?',
    Parameters=[{'S': 'Data Science'}]
)
for item in response['Items']:
    print(f"  {item['name']['S']} — GPA: {item['gpa']['N']}")

# UPDATE with PartiQL
print('\n=== PartiQL: Update Bob\'s year ===')
client.execute_statement(
    Statement=f'UPDATE "{TABLE_NAME}" SET year=? WHERE student_id=?',
    Parameters=[{'N': '4'}, {'S': 'stu-002'}]
)
# Verify the update
resp = client.execute_statement(
    Statement=f'SELECT name, year FROM "{TABLE_NAME}" WHERE student_id=?',
    Parameters=[{'S': 'stu-002'}]
)
for item in resp['Items']:
    print(f"  {item['name']['S']} is now year {item['year']['N']}")

## Step 7: DynamoDB → Pandas DataFrame

As data scientists, you want your data in DataFrames. Here are two methods:
- **Manual:** Scan → convert Decimals → `pd.DataFrame`
- **awswrangler:** PartiQL query → DataFrame in one line

In [ ]:
import pandas as pd

# ---- Method 1: Manual scan → DataFrame ----
response = table.scan()

def decimals_to_floats(item):
    """DynamoDB returns Decimal; Pandas wants float."""
    return {k: float(v) if isinstance(v, Decimal) else v
            for k, v in item.items()}

items = [decimals_to_floats(i) for i in response['Items']]
df = pd.DataFrame(items)

print('=== Manual Scan → DataFrame ===')
print(df[['student_id', 'name', 'major', 'gpa']].to_string(index=False))
print(f'\nMean GPA: {df["gpa"].mean():.2f}')
print(f'\nGPA by major:')
print(df.groupby('major')['gpa'].mean())

In [ ]:
# ---- Method 2: awswrangler (AWS SDK for pandas) ----
!pip install -q awswrangler

import awswrangler as wr

# PartiQL query → DataFrame in ONE line
print('=== awswrangler: Full table ===')
df_wr = wr.dynamodb.read_partiql_query(
    query=f'SELECT * FROM "{TABLE_NAME}"',
    boto3_session=boto3.Session(region_name=REGION)
)
print(df_wr)

print('\n=== awswrangler: Filtered query ===')
ds_df = wr.dynamodb.read_partiql_query(
    query=f'SELECT name, gpa FROM "{TABLE_NAME}" WHERE major=?',
    parameters=['Data Science'],
    boto3_session=boto3.Session(region_name=REGION)
)
print(ds_df)

## Step 8: Delete

In [ ]:
print('Deleting David (stu-004)...')
table.delete_item(Key={'student_id': 'stu-004'})
resp = table.get_item(Key={'student_id': 'stu-004'})
print(f'Confirmed deleted: {"Item" not in resp}')

print(f'\nRemaining: {len(table.scan()["Items"])} items')

## Step 9: Mini Challenge
1. Add 2+ new student records with **unique attributes** (e.g., `club`, `graduation_date`, `internship` map)
2. Use **PartiQL** to query for students matching criteria you choose
3. Load all items into a DataFrame and compute a **summary statistic** (mean, count, etc.)
4. **Screenshot your output** for Canvas submission

In [ ]:
# YOUR CODE: Add new students with unique attributes



In [ ]:
# YOUR CODE: PartiQL query



In [ ]:
# YOUR CODE: Load into DataFrame and compute a stat



## Cleanup
**Run this when done** to delete the table and avoid charges.

In [ ]:
print(f'Deleting table "{TABLE_NAME}"...')
table.delete()
table.wait_until_not_exists()
print('Table deleted. Verify in AWS Console that no tables remain.')